In [2]:
import numpy as np
import pandas as pd
from astropy.io import fits
import glob
from collections import defaultdict

In [107]:
# Read in the field information

root = "/home/esmee/Documents/DBDM/DDM2017-master/FinalProject/" #"/disks/strw9/stoop/DBDM/DDM2017-master/FinalProject/Q1/Tables/"
field_info = pd.read_csv(root + 'file_info_for_problem.csv')
field_info.insert(column="StarID", loc=4, value = 0)
print field_info

    ID  FieldID                              Filename Filter  StarID  \
0    1        1    Z-ADP.2017-01-18T11:58:36.905.fits      Z       0   
1    2        1    J-ADP.2017-01-18T11:58:35.781.fits      J       0   
2    3        1    H-ADP.2017-01-18T11:58:35.780.fits      H       0   
3    4        1   Ks-ADP.2016-05-25T15:33:39.546.fits     Ks       0   
4    5        1   Ks-ADP.2017-01-18T11:58:39.907.fits     Ks       0   
5    6        1   Ks-ADP.2016-05-25T15:33:43.377.fits     Ks       0   
6    7        1    Y-ADP.2017-01-18T11:58:36.901.fits      Y       0   
7    8        2   Z-ADP.2017-01-18T11:58:36.905b.fits      Z       0   
8    9        2   J-ADP.2017-01-18T11:58:35.781b.fits      J       0   
9   10        2   H-ADP.2017-01-18T11:58:35.780b.fits      H       0   
10  11        2  Ks-ADP.2016-05-25T15:33:39.546b.fits     Ks       0   
11  12        2   Y-ADP.2017-01-18T11:58:36.901b.fits      Y       0   
12  13        3   Z-ADP.2017-01-18T11:58:36.905c.fits      Z    

In [108]:
# Read in the different fits files
fnames = glob.glob(root + '*.fits')

#Find the number of unique stars
starIDs = []
for i, f in enumerate(fnames):
    hdu = fits.open(f)
    starIDs.append([dat for dat in hdu[1].data['StarID']])

nstars = len(np.unique(starIDs))
starCols = np.array(['ID', 'StarID', 'Ra', 'Dec', 'Flux1_Z', 'Flux1_Y', 'Flux1_J', 'Flux1_H', 'Flux1_Ks-E001',
       'Flux1_Ks-E002', 'Flux1_Ks-E003', 'Flux2_Z', 'Flux2_Y', 'Flux2_J',
       'Flux2_H', 'Flux2_Ks-E001', 'Flux2_Ks-E002', 'Flux2_Ks-E003',
       'Flux3_Z', 'Flux3_Y', 'Flux3_J', 'Flux3_H', 'Flux3_Ks-E001',
       'Flux3_Ks-E002', 'Flux3_Ks-E003', 'dFlux1_Z', 'dFlux1_Y',
       'dFlux1_J', 'dFlux1_H', 'dFlux1_Ks-E001', 'dFlux1_Ks-E002',
       'dFlux1_Ks-E003', 'dFlux2_Z', 'dFlux2_Y', 'dFlux2_J', 'dFlux2_H',
       'dFlux2_Ks-E001', 'dFlux2_Ks-E002', 'dFlux2_Ks-E003', 'dFlux3_Z',
       'dFlux3_Y', 'dFlux3_J', 'dFlux3_H', 'dFlux3_Ks-E001',
       'dFlux3_Ks-E002', 'dFlux3_Ks-E003', 'Mag1_Z', 'Mag1_Y', 'Mag1_J',
       'Mag1_H', 'Mag1_Ks-E001', 'Mag1_Ks-E002', 'Mag1_Ks-E003', 'Mag2_Z',
       'Mag2_Y', 'Mag2_J', 'Mag2_H', 'Mag2_Ks-E001', 'Mag2_Ks-E002',
       'Mag2_Ks-E003', 'Mag3_Z', 'Mag3_Y', 'Mag3_J', 'Mag3_H',
       'Mag3_Ks-E001', 'Mag3_Ks-E002', 'Mag3_Ks-E003', 'dMag1_Z',
       'dMag1_Y', 'dMag1_J', 'dMag1_H', 'dMag1_Ks-E001', 'dMag1_Ks-E002',
       'dMag1_Ks-E003', 'dMag2_Z', 'dMag2_Y', 'dMag2_J', 'dMag2_H',
       'dMag2_Ks-E001', 'dMag2_Ks-E002', 'dMag2_Ks-E003', 'dMag3_Z',
       'dMag3_Y', 'dMag3_J', 'dMag3_H', 'dMag3_Ks-E001', 'dMag3_Ks-E002',
       'dMag3_Ks-E003']) 

fmcols = ['Flux', 'dFlux', 'Mag', 'dMag']
uniq_IDs = np.unique(starIDs)
nstars = len(np.unique(starIDs))
ncols = 5 + 83
print "Table is {0} x {1}".format(nstars, ncols)
stable = np.zeros((nstars, ncols))
starTable = pd.DataFrame(data=stable, columns=starCols)

Table is 30000 x 88



# The database tables

**FieldTable**

ID | FieldID | Filename | Filter | Exp time | MJD | StarIDs |


**StarTable**

ID  | StarID  | Ra  | Dec  | Flux1_Z  | dFlux1_Z  | ..  | Flux3_Ks  | dFlux3_Ks  | Mag1_Z  | dMag1_Z  | ..  | Mag3_Ks  | dMag3_Ks  | Var_Ks |  



In [128]:
IDs_seen = []
fmcols = ['Flux', 'dFlux', 'Mag', 'dMag']

for f in fnames:
    print "Started with file {0}".format(f.replace(root, ''))
    
    hdu = fits.open(f)
    f = f.replace(root, '')
    sID = hdu[1].data['StarID']
    ind = [np.where(uniq_IDs == i)[0][0] for i in sID]
    filt = f[8:-5]
    
    #Send star IDs to the field_info
    fID = f[6] #field
    if 'Ks' in filt:
        filt2 = 'Ks'
        name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt2)]))
        name = name[int(filt[-1])-1]
    else:
        name = np.sort(np.array(field_info["Filename"][(field_info["FieldID"] == int(fID)) & (field_info["Filter"] == filt)]))
        name = name[0]
    field_info["StarID"][np.where(field_info["Filename"] == name)[0][0]] = np.array_str(sID)
    
    seen = [ID in IDs_seen for ID in sID]

    # Is iD star already seen?
    if not seen[0]:
        starTable['ID'][ind] = ind
        # Input the StarID, Ra and Dec
        for c in starCols[1:4]:
            starTable[c][ind] = hdu[1].data[c]

        [IDs_seen.append(ID) for ID in sID]
        
        # Input the flux and magnitude of this data
        for fm in fmcols:
            for t in range(1, 4):
                starTable[fm + str(t) +'_' + filt][ind] = hdu[1].data[fm + str(t)]

    # Go through all other flux, mag and filters
    else:
        for fm in fmcols:
            for t in range(1, 4):
                starTable[fm + str(t) +'_' + filt][ind] = hdu[1].data[fm + str(t)]
    

    print "Finished"

Started with file Field-1-Z.fits


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Started with file Field-1-Y.fits
Started with file Field-3-Z.fits
Started with file Field-3-H.fits
Started with file Field-3-Y.fits
Started with file Field-1-H.fits
Started with file Field-1-Ks-E003.fits
Started with file Field-1-Ks-E001.fits
Started with file Field-3-J.fits
Started with file Field-2-Ks-E001.fits
Started with file Field-2-Z.fits
Started with file Field-2-J.fits
Started with file Field-3-Ks-E001.fits
Started with file Field-2-H.fits
Started with file Field-2-Y.fits
Started with file Field-1-J.fits
Started with file Field-3-Ks-E002.fits
Started with file Field-1-Ks-E002.fits
Finished


In [131]:
# write tables to csv
starTable.to_csv(root+'starTable.csv')
field_info.to_csv(root+'fieldTable.csv')